In [2]:
from gekko import GEKKO
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

m = GEKKO() # initialize GEKKO
nt = 501
m.time = np.linspace(0,1,nt)
# Variables  ## initial value
x1 = m.Var(value=0)
x2 = m.Var(value=0)
# x3 = m.Var(value=0.0)
p = np.zeros(nt) # final time = 1
p[-1] = 1.0
final = m.Param(value=p)
# optimize final time
tf = m.FV(value=1.0,lb=0.01,ub=100.0)
tf.STATUS = 1
# control changes every time period
u = m.MV(value=0,lb=-10,ub=10) ## control value
u.STATUS = 1
m.Equation(x1.dt()==x2*tf)
m.Equation(x2.dt()==u*tf)
# m.Equation(x3.dt()==m.sin(x1)*tf)
m.Equation(x1*final>=10)
m.Equation(x2*final<=0)
m.Obj(tf)
m.options.IMODE = 6
m.solve(disp=False)
print('Final Time: ' + str(tf.value[0]))
tm = np.linspace(0,tf.value[0],nt)
plt.figure(1)
plt.plot(tm,x1.value,'k-',label=r'$x_1$')
plt.plot(tm,x2.value,'b-',label=r'$x_2$')
# plt.plot(tm,x3.value,'g--',label=r'$x_3$')
plt.plot(tm,u.value,'r--',label=r'$u$')
plt.legend(loc='best')
plt.xlabel('Time')

Exception:  @error: Solution Not Found


In [9]:
## https://apmonitor.com/do/index.php/Main/RocketOptimization

import numpy as np
import matplotlib.pyplot as plt
from gekko import GEKKO

# create GEKKO model
m = GEKKO()

# scale 0-1 time with tf
m.time = np.linspace(0,1,1001)

# options
m.options.NODES = 6
m.options.SOLVER = 3
m.options.IMODE = 6
m.options.MAX_ITER = 5000
m.options.MV_TYPE = 0
m.options.DIAGLEVEL = 0

# final time
tf = m.FV(value=1.0,lb=0.1,ub=15)
tf.STATUS = 1

# force
u = m.MV(value=10,lb=-10,ub=10)
u.STATUS = 1
u.DCOST = 1e-5

# variables
s = m.Var(value=0,lb=0, ub=100)
v = m.Var(value=0,lb=-10,ub=4)
## mass = m.Var(value=1,lb=0.2)

# differential equations scaled by tf
m.Equation(s.dt()==tf*v)
m.Equation(v.dt()==tf*(u))
# m.Equation(mass.dt()==tf*(-0.01*u**2))

# specify endpoint conditions
m.fix(s, pos=len(m.time)-1,val=10.0)
#m.fix(v, pos=len(m.time)-1,val=0.0)

# minimize final time
m.Obj(tf)

# Optimize launch
m.solve()

print('Optimal Solution (final time): ' + str(tf.value[0]))

# scaled time
ts = m.time * tf.value[0]

# plot results
plt.figure(1)
plt.subplot(3,1,1)
plt.plot(ts,s.value,'r-',lw=2)
plt.ylabel('Position')
plt.legend(['s (Position)'])

plt.subplot(3,1,2)
plt.plot(ts,v.value,'b-',lw=2)
plt.ylabel('Velocity')
plt.legend(['v (Velocity)'])

# plt.subplot(4,1,3)
# plt.plot(ts,mass.value,'k-',lw=2)
# plt.ylabel('Mass')
# plt.legend(['m (Mass)'])

plt.subplot(3,1,3)
plt.plot(ts,u.value,'g-',lw=2)
plt.ylabel('Force')
plt.legend(['u (Force)'])

plt.xlabel('Time')
plt.show()

apm 118.201.101.210_gk_model7 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            4
   Intermediates:            0
   Connections  :            2
   Equations    :            3
   Residuals    :            3
 
 Number of state variables:          28999
 Number of total equations: -        28000
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :            999
 
 ----------------------------------------------
 Dynamic Control with APOPT Solver
 ----------------------------------------------


In [5]:
def get_theory_T(p): 
    return 2.5 + 40 /(100 - p ** 2)

print (get_theory_T(0))

2.9
